In [1]:
import multiprocessing as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
basedir = './국토교통부_건축물대장_층별개요/'
filenames = [f for f in os.listdir(basedir) if (f.endswith('.csv'))&(f.startswith('floor'))]
filenames

['floor_info_baseline_seoul_201412.csv',
 'floor_info_baseline_seoul_201512.csv',
 'floor_info_baseline_seoul_201612.csv',
 'floor_info_baseline_seoul_201712.csv',
 'floor_info_baseline_seoul_201812.csv',
 'floor_info_baseline_seoul_201912.csv',
 'floor_info_baseline_seoul_202011.csv']

In [3]:
dfs_list = []
for file in filenames:
    df = pd.read_csv(basedir + file)
    df['기준년월'] = int(file.replace('.csv','')[-6:])
    dfs_list.append(df)

c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (10,11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
dfs_list[0] = dfs_list[0].rename(columns={'건축물대장_PK':'관리_건축물대장_PK', '블럭':'블록', '동_명칭':'동_명', '면적':'면적(㎡)'})

In [6]:
dfs_list[0] = dfs_list[0].drop(columns=['마트_층별_개요_PK', '지번_좌표_X', '지번_좌표_Y', '도로명_좌표_X', '도로명_좌표_Y',
                                        '기타_기재_사항'
                                       ])

In [7]:
first_cols = dfs_list[0].columns

In [8]:
for df in dfs_list:
    print(first_cols == df.columns)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  Tr

In [9]:
concat_df = pd.concat(dfs_list).reset_index(drop=True)

In [10]:
concat_df.drop(columns=[col for col in concat_df.columns if col.endswith('_코드')], inplace=True)
concat_df.shape

(18712608, 24)

In [11]:
concat_df.columns = [col.replace('_코드_명', '') for col in concat_df.columns]
concat_df.head()

,관리_건축물대장_PK,대지_위치,도로명_대지_위치,건물_명,번,지,특수지_명,블록,로트,새주소_본_번,새주소_부_번,동_명,층_구분,층_번호,층_번호_명,구조,기타_구조,주_용도,기타_용도,면적(㎡),주_부속_구분,면적_제외_여부,생성_일자,기준년월
0,11440-3279202,서울특별시 마포구 중동 81-29번지,NaN,중동빌라,81.0,29.0,NaN,NaN,NaN,9.0,6.0,NaN,지상,1,1층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,57.62,주건축물,NaN,20141109,201412
1,11440-3279202,서울특별시 마포구 중동 81-29번지,NaN,중동빌라,81.0,29.0,NaN,NaN,NaN,9.0,6.0,NaN,지상,2,2층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,67.20,주건축물,NaN,20141109,201412
2,11440-3279202,서울특별시 마포구 중동 81-29번지,NaN,중동빌라,81.0,29.0,NaN,NaN,NaN,9.0,6.0,NaN,지상,3,3층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,67.20,주건축물,NaN,20141109,201412
3,11440-3279202,서울특별시 마포구 중동 81-29번지,NaN,중동빌라,81.0,29.0,NaN,NaN,NaN,9.0,6.0,NaN,지상,4,4층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,59.68,주건축물,NaN,20141109,201412
4,11440-3279202,서울특별시 마포구 중동 81-29번지,NaN,중동빌라,81.0,29.0,NaN,NaN,NaN,9.0,6.0,NaN,옥탑,1,옥탑 1층,철근콘크리트구조,철근콘크리트구조,다세대주택,계단실(연면적제외),7.68,주건축물,NaN,20141109,201412


In [12]:
def correct_addr(x):
    # x is a string
    if (pd.isna(x) == True)|(x == '없음'):
        return x
    else:
        x = x.replace('번지', '')
        if x[0] == ' ':
            for i in range(len(x)):
                if x[i] != ' ':
                    x = x[i:]
                    break
        
        splitted = x.split('-')
        if len(splitted) == 1:
            return x+'-0'
        else:
            return x

In [13]:
concat_df['지번주소'] = concat_df['대지_위치'].apply(correct_addr)

In [14]:
concat_df.drop(columns=['대지_위치', '도로명_대지_위치', '번', '지', '특수지_명', '블록', '로트', '새주소_본_번', '새주소_부_번',
                        '동_명'
                       ], inplace=True)
concat_df.head()

,관리_건축물대장_PK,건물_명,층_구분,층_번호,층_번호_명,구조,기타_구조,주_용도,기타_용도,면적(㎡),주_부속_구분,면적_제외_여부,생성_일자,기준년월,지번주소
0,11440-3279202,중동빌라,지상,1,1층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,57.62,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29
1,11440-3279202,중동빌라,지상,2,2층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,67.20,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29
2,11440-3279202,중동빌라,지상,3,3층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,67.20,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29
3,11440-3279202,중동빌라,지상,4,4층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,59.68,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29
4,11440-3279202,중동빌라,옥탑,1,옥탑 1층,철근콘크리트구조,철근콘크리트구조,다세대주택,계단실(연면적제외),7.68,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29


In [15]:
concat_df['addr_split_len'] = concat_df['지번주소'].apply(lambda x: len(x.split(' ')))

In [16]:
concat_df['addr_split_len'].value_counts()

4    18624900
5       65236
6       17162
7        4197
8         730
3         291
9          92
Name: addr_split_len, dtype: int64

In [19]:
concat_df = concat_df[(concat_df['addr_split_len'] == 4)]
concat_df['addr_split_len'].value_counts()

4    18624900
Name: addr_split_len, dtype: int64

In [20]:
def weird_addr(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        try:
            int(x.split(' ')[-1].replace('-', ''))
            return 0
        except:
            return 1

In [21]:
len4_df = concat_df[concat_df['addr_split_len'] == 4]
len4_df['weird_addr'] = len4_df['지번주소'].apply(weird_addr)

In [22]:
weird_addr_df = len4_df[len4_df['weird_addr'] == 1]
weird_addr_df.shape

(3742, 17)

In [23]:
concat_df.drop(weird_addr_df.index, inplace=True)

In [24]:
concat_df.shape

(18621158, 16)

In [25]:
main_data_df = pd.read_csv('./prepped_data/main_data_baseline_checkpoint_2.csv')
print(main_data_df.shape)
main_data_df.head()

(4441256, 17)


,전월세매매구분,전용면적(㎡),계약일,금액(만원),월세(만원),층,건축년도,가격/면적,년,월,target_log_transformed,target/area_log_transformed,건물종류,지번주소,도로명주소,X좌표,Y좌표
0,매매,77.75,8,57000,0,2,1988,733.118971,2013,9,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06
1,매매,77.75,16,57000,0,2,1988,733.118971,2013,12,10.950807,6.597308,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06
2,매매,77.75,29,55000,0,7,1988,707.395498,2014,1,10.915088,6.561590,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06
3,매매,77.75,10,59500,0,4,1988,765.273312,2014,11,10.993732,6.640233,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06
4,매매,77.75,28,75000,0,5,1988,964.630225,2016,3,11.225243,6.871745,아파트,서울특별시 강남구 개포동 655-2,서울특별시 강남구 언주로 103-0,960473.743425,1.942553e+06


In [26]:
inter_df = concat_df[concat_df['지번주소'].isin(main_data_df['지번주소'].unique())]
print(inter_df.shape)
inter_df.head()

(6880841, 16)


,관리_건축물대장_PK,건물_명,층_구분,층_번호,층_번호_명,구조,기타_구조,주_용도,기타_용도,면적(㎡),주_부속_구분,면적_제외_여부,생성_일자,기준년월,지번주소,addr_split_len
0,11440-3279202,중동빌라,지상,1,1층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,57.62,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4
1,11440-3279202,중동빌라,지상,2,2층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,67.20,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4
2,11440-3279202,중동빌라,지상,3,3층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,67.20,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4
3,11440-3279202,중동빌라,지상,4,4층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,59.68,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4
4,11440-3279202,중동빌라,옥탑,1,옥탑 1층,철근콘크리트구조,철근콘크리트구조,다세대주택,계단실(연면적제외),7.68,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4


In [27]:
%%time
inter_df.to_csv('floor_info_maindata_inter_baseline_2.csv', index=False)

Wall time: 38.7 s
